## PPO
TRPO由于计算过程过于复杂，因此有改进版本PPO算法，基于TRPO而且比TRPO学习锝更好，PPO首先属于可以尝试的算法。
<br>回顾TRPO的优化目标：
$$
\begin{aligned} \max _{\theta^{\prime}} & L_{\theta}\left(\theta^{\prime}\right) \\ \text { s.t. } & \mathbb{E}_{s \sim \nu^{\pi_{\theta}}}\left[D_{K L}\left(\pi_{\theta_{k}}(\cdot \mid s), \pi_{\theta^{\prime}}(\cdot \mid s)\right)\right] \leq \delta\end{aligned}
$$
TRPO 使用泰勒展开近似、共轭梯度、线性搜索等方法直接求解。PPO 的优化目标与 TRPO 相同，但 PPO 用了一些相对简单的方法来求解。具体来说，PPO 有两种形式，一是 PPO-惩罚，二是 PPO-截断

### PPO-惩罚
PPO-惩罚（PPO-Penalty）用拉格朗日乘数法直接将 KL 散度的限制放进了目标函数中，这就变成了一个无约束的优化问题，在迭代的过程中不断更新 KL 散度前的系数：
$$
\underset{\theta}{\arg \max } \mathbb{E}_{s \sim \nu}^{\pi_{\theta_{k}}} \mathbb{E}_{a \sim \pi_{\theta_{k}}(\cdot \mid s)}\left[\frac{\pi_{\theta}(a \mid s)}{\pi_{\theta_{k}}(a \mid s)} A^{\pi_{\theta_{k}}}(s, a)-\beta D_{K L}\left[\pi_{\theta_{k}}(\cdot \mid s), \pi_{\theta}(\cdot \mid s)\right]\right]
$$
令$d_{k}=D_{K L}^{\nu^{\pi_{\theta_{k}}}}\left(\pi_{\theta_{k}}, \pi_{\theta}\right)$ ， $\beta$ 的更新规则如下:
1. 如果 $d_{k}<\delta / 1.5$ ，那么 $\beta_{k+1}=\beta_{k} / 2$
2. 如果 $d_{k}>\delta \times 1.5$ ，那么 $\beta_{k+1}=\beta_{k} \times 2$
3. 否则 $\beta_{k+1}=\beta_{k}$
其中$\delta$是事先设定的超参数，用于限制学习策略和之前一轮策略的差距

### PPO-截断
PPO截断更加直接，其实就是确保新参数和旧参数位于$1-\epsilon$和$1+\epsilon$之间：
$$
\underset{\theta}{\arg \max } \mathbb{E}_{s \sim \nu}{ }_{s \theta_{k}} \mathbb{E}_{a \sim \pi_{\theta_{k}}(\cdot \mid s)}\left[\min \left(\frac{\pi_{\theta}(a \mid s)}{\pi_{\theta_{k}}(a \mid s)} A^{\pi_{\theta_{k}}}(s, a), \operatorname{clip}\left(\frac{\pi_{\theta}(a \mid s)}{\pi_{\theta_{k}}(a \mid s)}, 1-\epsilon, 1+\epsilon\right) A^{\pi_{\theta_{k}}}(s, a)\right)\right]
$$
其中$\operatorname{clip}(x, l, r):=\max (\min (x, r), l)$，即把x限制在[l,r]之间，$\epsilon$是一个超参数，表示截断的范围
#### 代码实现：
截断主要是通过`torch.clamp()`方法实现：
```python
surr1 = ratio * advantage
surr2 = torch.clamp(ratio, 1 - self.eps,1 + self.eps) * advantage # 截断
actor_loss = torch.mean(-torch.min(surr1, surr2))  # PPO损失函数
```

### 代码实现
> 在14-BC-RL-GAIL笔记中，混合实现完整PPO

#### 注意：连续动作空间使用高斯分布，注意休怪